In [2]:
import numpy as np
from bs4 import BeautifulSoup

data = open("/home/user/Downloads/UKWAC-1_new.xml",'r', encoding='latin5').read()
bs_data = BeautifulSoup(data, 'xml')
del data

In [ ]:
import pickle
pickle.dump(bs_data, open("UKWAC_1_BS4.pickle", "wb"))

In [1]:
texts = bs_data.find_all("text")
for i in range(len(texts)):
    texts[i] = texts[i].find_all("s")
    
for i in range(len(texts)):
    for j in range(len(texts[i])):
        print(texts[i][j].text)
        break
    break

    

NameError: name 'bs_data' is not defined